In [1]:
import csv
import os
import numpy as np
import cv2
import keras

from keras.models import load_model
from keras.models import model_from_json
import h5py
from keras import __version__ as keras_version

from time import sleep
from tqdm import tqdm

dataset_path = "../../data/speed_challenge_2017/data/test_seq/"
output_path = "../../data/speed_challenge_2017/data/train.txt"

# with open(output_path, 'r') as h:
#     lines = h.readlines()

image_paths = sorted(os.listdir(dataset_path))

dataset = []
for i in range(len(image_paths)):
    #speed_val = float(lines[i].strip())
    speed_val = 0.0
    dataset.append({"image": image_paths[i], "speed": speed_val})

print(len(dataset))
print(dataset[5])

Using TensorFlow backend.


10798
{'image': 'out00006.png', 'speed': 0.0}


In [3]:
seq_images = []
seq_len = 10

def img_for_model(data):
    img = cv2.imread(dataset_path + data["image"], 0)[30:370, :]
    img = cv2.equalizeHist(img)
    img = np.asarray(img).reshape(340, 640, 1)
    return img

def predict_speed(model, data):
    global seq_images, seq_len

    pred_speed = 0.0
    gt_speed = data["speed"]

    image_array = img_for_model(data)

    if len(seq_images) < seq_len:
        seq_images.append(image_array)

        transformed_image_array = np.array(seq_images)
        transformed_image_array = transformed_image_array[None, :, :, :, :]

        pred_speed = float(model.predict(transformed_image_array, batch_size=1))*15 + 15


    else:
        seq_images.pop(0)
        seq_images.append(image_array)

        transformed_image_array = np.array(seq_images)
        transformed_image_array = transformed_image_array[None, :, :, :, :]

        pred_speed = float(model.predict(transformed_image_array, batch_size=1))*15 + 15

    return pred_speed, gt_speed

In [4]:
# compile and load weights
model_path = "../../data/weights/speed_challenge/model.json"
with open(model_path, 'r') as jfile:
    model = model_from_json(jfile.read())

model.compile("rmsprop", "mse")
weights_file = model_path.replace('json', 'h5')
model.load_weights(weights_file)

In [ ]:

font = cv2.FONT_HERSHEY_SIMPLEX
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('comma_test.avi',fourcc, 20.0, (640,480))

for i in tqdm(range(len(dataset)), total=len(dataset)):
    pred_speed, gt_speed = predict_speed(model, dataset[i])
    pred_speed = round(pred_speed, 2)
    gt_speed = round(gt_speed, 2)
    image = cv2.imread(dataset_path + dataset[i]["image"])
    cv2.putText(image,str(pred_speed), (20,420), font, 0.5,(0,255,0),1,cv2.LINE_AA)
    cv2.putText(image,str(gt_speed), (20,450), font, 0.5,(0,0,255),1,cv2.LINE_AA)
    out.write(image)
#     cv2.imshow("Front view", image)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#        break

out.release()
#cv2.destroyAllWindows()

 25%|██▌       | 2746/10798 [01:04<03:05, 43.35it/s]